In [17]:
# import requests
import requests
import json

In [18]:
# geo.admin lookup

def get_lat_lon_wgs84(address):
    url = f"https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText={address}&type=locations&origins=address"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["lat"], data["results"][0]["attrs"]["lon"]

get_lat_lon_wgs84("St. Leonhard-Strasse 45, St. Gallen")

(47.42184829711914, 9.369429588317871)

In [24]:
def get_easting_northing_lv95(address):

    url = f"https://api3.geo.admin.ch//rest/services/api/SearchServer?lang=de&searchText={address}&type=locations&sr=2056"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["y"], data["results"][0]["attrs"]["x"]

get_easting_northing_lv95("St. Leonhard-Strasse 45, St. Gallen")

(2745681.0, 1254130.25)

In [25]:
# function to get heating info from long and lat

def get_heating_info(lat, lon, dist):
    url = f"https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=warmeversorgung&q=&facet=waermevers&geofilter.distance={lat}%2C{lon}%2C{dist}"
    response = requests.get(url)
    data = response.json()
    return data

# function to get heating info from address

def get_heating_info_from_address(address, dist):
    lat, lon = get_lat_lon_wgs84(address)
    return get_heating_info(lat, lon, dist)

get_heating_info_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

{'nhits': 1,
 'parameters': {'dataset': 'warmeversorgung',
  'rows': 10,
  'start': 0,
  'facet': ['waermevers'],
  'format': 'json',
  'geofilter.distance': ['47.42184829711914,9.369429588317871,1'],
  'timezone': 'UTC'},
 'records': [{'datasetid': 'warmeversorgung',
   'recordid': '41a0060d5f24a87f0c0f390b4e363916bd084a21',
   'fields': {'geo_shape': {'coordinates': [[[9.369402092347727,
        47.42141803157732],
       [9.36880080458217, 47.421178234079115],
       [9.36845445807961, 47.42153135919113],
       [9.369074349371125, 47.421789138301264],
       [9.369639976357611, 47.42203562120369],
       [9.369902089204528, 47.42170847062191],
       [9.369460130185418, 47.421532842160765],
       [9.369402092347727, 47.42141803157732]]],
     'type': 'Polygon'},
    'waermevers': 'Fernwärme Ergänzungsgebiet',
    'geo_point_2d': [47.42160099351865, 9.369180665597083],
    'dist': '33.27289013497016'},
   'geometry': {'type': 'Point',
    'coordinates': [9.369180665597083, 47.42160

In [26]:
# function to get waermevers from address

def get_waermevers_from_address(address, dist):
    data = get_heating_info_from_address(address, dist)
    return data["records"][0]["fields"]["waermevers"]

get_waermevers_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

'Fernwärme Ergänzungsgebiet'

# data from geo.admin.ch

ch.bfe.solarenergie-eignung-daecher
ch.bfe.solarenergie-eignung-fassaden

# get data from geo.admin.ch




In [28]:
# https://github.com/ThesisRocker/Data2DollarFS22/blob/0f7344650e6c6c43bc4057502493ea4afbce64f3/06_Abgabe%20Gruppenarbeit/Gruppe%201/sonnendach/script_solar_2.py

def get_geo_admin_data(address):
    addr_easting, addr_northing = get_easting_northing_lv95(address)

    # Build the request similar to script_solar only with new coordinates from previous request
    # Uses the swiss coordinate system LV95
    API_building_ID = ('https://api3.geo.admin.ch//rest/services/api/MapServer/identify'
                       '?geometryType=esriGeometryPoint&returnGeometry=true'
                       '&layers=all:ch.bfe.solarenergie-eignung-daecher'
                       '&geometry=' + str(addr_easting) + ',' + str(addr_northing) +
                       '&tolerance=0&order=distance&lang=de&sr=2056')

    # Make the request to the API using requests library
    result_building = requests.get(API_building_ID)

    # Transform result into json format
    result_building_json = result_building.json()

    print(result_building_json)

get_geo_admin_data("St. Leonhard-Strasse 45, St. Gallen")

{'results': [{'featureId': 16745798, 'bbox': [2745661.8, 1254117.1, 2745696.4, 1254140.8], 'layerBodId': 'ch.bfe.solarenergie-eignung-daecher', 'layerName': 'Solarenergie: Eignung Dächer', 'id': 16745798, 'geometry': {'rings': [[[2745693.9, 1254130.9], [2745671.7, 1254117.1], [2745661.8, 1254119.4], [2745696.4, 1254140.8], [2745693.9, 1254130.9]]], 'spatialReference': {'wkid': 2056}}, 'attributes': {'monate': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 12, 11], 'building_id': 527069, 'a_param': [0.2906374447, 0.611679594, 0.7083464405, 0.7987807161, 0.7741414084, 0.7471578003, 0.6675143136, 0.5009496253, 0.2478713579, -0.0245813054, -0.0604781153, 0.0667746012], 'b_param': [0.7131205661, 0.8232165751, 0.7035129153, 0.9269735806, 0.8109882235, 0.839912098, 0.8305562297, 0.8556729308, 0.8443842721, 0.7968375558, 0.7635561032, 0.8234379675], 'c_param': [6.3315561626, -0.5449509976, 11.8673678461, -2.8281753894, 9.8337065283, 4.8559245077, 2.5344731962, -0.3054260927, -0.34722585, 1.26600546, 1.206596